<hr style="color:green" />
<h1 style="color:green">COSC2673 Assignment 2: Image Classification for Cancerous Cells</h1>
<h2 style="color:green">File 31: Modelling on the Full data For the Cell Type Multiclass Classification</h2>
<hr style="color:green" />

<p>
After Reviewing files 22, 24, 25 and 26, we have experimented with improving the performance of the CNN with more complexity in the Convolutions and the Classifier layers. Taking into account performance increases but also processing time and complexity, the following model will be used here:
</p>
<ul>
<li>CNN with more complex convolutions (from file 24)</li>
</ul>
<p>
Now, in an attempt to improve the performance of both the Binary and the Multiclass models, this process will make use of the full data, ie. both the Main Labels file and the Extras Label file. Because handling these two will be different, processing will be split into separate notebooks.
</p>

<ul>
<li>Binary IsCancerous: All data is labeled, so just add the new data to the dataset</li>
<li>Multiclass Cell Type (here): The data in the Extras file are unlabeled, so therefore, we will need to apply a Semi Supervised Learning approach</li>
</ul>
<p>
In this file, also load the Extra Label data as well as the main data. Then we will perform a Semi Supervised learning process. The basic premise of Semi Supervised learning will be  as follows
</p>

<ol>
<li>Train a baseline model using the labelled Main Data</li>
<li>Predict using this model on the unlabelled Extra Data, taking care to also retain the Softmax score for each value, which is a probability of being that label</li>
<li>Review these predictions, find any that have a high confidence (generally 80% probability is used)</li>
<li>Take this data, and apply that predicted label, called a "pseudo-label". Incorporate this pseudo-labeled data into the training data (also remove them from the extra data)</li>
<li>Train an updated model using the updated training data</li>
<li>Iterate the process from Step 2, until no high confidence data is generated, or capped at a number of rounds</li>
</ol>

In [1]:
import pandas as pd
import numpy as np
import os
import cv2

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.utils.data
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import read_image


c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Configure this script as to whether it runs on Google Colab, or locally

In [2]:
# When on Google Colab, running full training, change both to true. Locally, advised set both to false
isGoogleColab = False
useFullData = False

In [3]:
# In local, the base directory is the current directory
baseDirectory = "./"

if isGoogleColab:
    from google.colab import drive
    
    # If this is running on Google colab, assume the notebook runs in a "COSC2673" folder, which also contains the data files 
    # in a subfolder called "image_classification_data"
    drive.mount("/content/drive")
    !ls /content/drive/'My Drive'/COSC2673/

    # Import the directory so that custom python libraries can be imported
    import sys
    sys.path.append("/content/drive/MyDrive/COSC2673/")

    # Set the base directory to the Google Drive specific folder
    baseDirectory = "/content/drive/MyDrive/COSC2673/"

Import the custom python files that contain reusable code

In [4]:
import data_basic_utility as dbutil
import graphing_utility as graphutil
import statistics_utility as statsutil

import a2_utility as a2util
import pytorch_utility as ptutil
from pytorch_utility import CancerBinaryDataset
from pytorch_utility import CancerCellTypeDataset


# randomSeed = dbutil.get_random_seed()
randomSeed = 266305
print("Random Seed: " + str(randomSeed))

Random Seed: 266305


In [5]:
# this file should have previously been created in the root directory
dfImages = pd.read_csv(baseDirectory + "images_main.csv")
dfImagesExtra = pd.read_csv(baseDirectory + "images_extra.csv")

In [6]:
# Get The training Split and the Validation Split
dfImagesTrain = dfImages[dfImages["trainValTest"] == 0].reset_index()
dfImagesVal = dfImages[dfImages["trainValTest"] == 1].reset_index()
dfImagesTest = dfImages[dfImages["trainValTest"] == 2].reset_index()

print(dfImagesTrain.shape)
print(dfImagesVal.shape)
print(dfImagesTest.shape)

dfImagesTrain.head()

(7837, 5)
(1031, 5)
(1028, 5)


,index,ImageName,isCancerous,cellType,trainValTest
0,0,./Image_classification_data/patch_images\1.png,0,0,0
1,1,./Image_classification_data/patch_images\10.png,0,0,0
2,3,./Image_classification_data/patch_images\1000.png,1,2,0
3,4,./Image_classification_data/patch_images\10000...,0,1,0
4,5,./Image_classification_data/patch_images\10001...,0,1,0


In the Semi Supervised Learning, there is no point in Splitting the Extra data into the flagged Train/Validation/Test Split, as it's unlabelled by Cell Type. Therefore, all extra data will be used as part of the SS Training process

Note: The definition of the Custom Datasets for both the isCancerous data and the Cell Type data are defined in the pytorch_utility.py file.

Also, rather than loading all the training images and calculating the mean and standard deviation values in here, that was run separately in file 05a.PyTorchGetMeanAndStd.ipynb

Here we can just define the values to use, which shouldn't change unless the data is reloaded and a new train/validation/test split is generated

In [7]:
train_mean, train_std = ptutil.getTrainMeanAndStdTensors()
print(train_mean)
print(train_std)

tensor([0.8035, 0.5909, 0.7640])
tensor([0.1246, 0.1947, 0.1714])


In [8]:
# Create a tranform operation that also normalizes the images according to the mean and standard deviations of the images
transform_normalize = transforms.Compose(
    [transforms.ToPILImage(),
    transforms.ToTensor(), 
    transforms.Normalize(train_mean, train_std)])


# Semi Supervised Learning

Initially, only create data loaders for the validation data and the test data, as these sets will not change. 

In [9]:
celltype_validation_data = CancerCellTypeDataset(isGoogleColab, dfImagesVal, baseDirectory, transform=transform_normalize)
celltype_test_data = CancerCellTypeDataset(isGoogleColab, dfImagesTest, baseDirectory, transform=transform_normalize)

# Create data loaders
celltype_val_dataloader = DataLoader(celltype_validation_data, batch_size=32, shuffle=True, num_workers=2)
celltype_test_dataloader = DataLoader(celltype_test_data, batch_size=32, shuffle=True, num_workers=2)

Define the classe for the CNN. At the moment, we are using the the standard CNN structure with Early stopping (File 22)

In [10]:
# Create a class for the Neural Network
class PT_CNN_CellType(nn.Module):

    # In the constructor, initialize the layers to use
    def __init__(self):
        super(PT_CNN_CellType, self).__init__()


        # first, define the subsampling methods. Though they are used multiple times, these are the
        # operations, so only need to be defined once
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        # define the Activation methods to use
        self.relu = nn.ReLU(inplace=True)
        self.softmax = nn.Softmax(dim=1)

        # define the convolution layers

        # input should be 27x27x3. Apply a 3x3 filter, therefore, output should be 25x25x32 (channels aka feature maps)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1)
        # There will be a Relu
        # Then a MaxPool of 2x2, halving the dimensions per feature map
        # So input is 12x12x32. Apply a 3x3 filter, also include padding=1, as this is already quite small, and lets consider the edges
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)    
        # There will be a Relu        
        # So input is 12x12x64. Apply a 3x3 filter, also include padding=1, as this is already quite small, and lets consider the edges
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        # Then a MaxPool of 2x2, halving the dimensions per feature map
        # Then an Average Pool of keeping the dimensions as 6x6
        
        # define the fully connected neural layers
        self.fc1 = nn.Linear(128 * 6 * 6, 4608)
        self.fc2 = nn.Linear(4608, 2306)
        self.fc3 = nn.Linear(2306, 4)

    # Create the forward function, which is used in training
    def forward(self, x):

        # print("Init Shape: " + str(x.shape))

        # Process the first 2 convolution layers, applying maxpooling
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)

        # Then process the remaining convolution layers without any pooling
        x = self.relu(self.conv2(x))        
        x = self.relu(self.conv3(x))

        # Then apply a max pool and average pool on the result
        x = self.maxpool(x)
        #x = self.avgpool(x)

        # Flatten: This should convert to tensors that are acceptable for the input into the NN 3 layers
        x = x.view(x.size(0), 128 * 6 * 6)

        # Now process the 3 layers of the Fully Connected NN
        x = self.relu(self.fc1(x))  
        x = self.relu(self.fc2(x))              
        # x = self.fc3(x)
        # x = self.relu(self.fc3(x))
        x = F.softmax(self.fc3(x), dim=1)        

        # return the result
        return x



Create a function to Predict using a particular mode and return results

In [11]:
def predictCellTypeOnDataSetAccuracy(net, setName, dataloader, printResult=True, printFullResults=False):
    correct, total = 0,  0
    predictions = []

    y_celltype = []
    y_pred_celltype = []
    y_pred_celltype_scores = []

    # Looping through this dataloader essentially processes them in batches of 32 (or whatever the batchsize is configured in the data loader
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data

        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        # Loop through the batch, build the lists of the raw label and prediction values
        for j in range(len(labels)):
            y_celltype.append(labels[j].item())
            y_pred_celltype.append(predicted[j].item())
            y_pred_celltype_scores.append(outputs.data[j].tolist())

        predictions.append(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = accuracy_score(y_celltype, y_pred_celltype)
    f1Score = f1_score(y_celltype, y_pred_celltype, average="micro")

    if printFullResults:
        print(setName + ":")        
        print('Confusion matrix: \n')
        print(confusion_matrix(y_celltype, y_pred_celltype))
        print("\n- Accuracy Score: " + str(accuracy))
        print("- Precision Score: " + str(precision_score(y_celltype, y_pred_celltype, average="micro")))
        print("- Recall Score: " + str(recall_score(y_celltype, y_pred_celltype, average="micro")))
        print("- F1 Score: " + str(f1Score))
    elif printResult:
        print("- " + setName + " F1: " + str(f1Score))

    return accuracy, y_celltype, y_pred_celltype, y_pred_celltype_scores

Set some standard learning variables that will be used across all iterations

In [12]:
# set the Learning Rate to use
learning_rate = 0.0001
maxEpochs = 15
patience = 2
disableEarlyStopping = False

if useFullData == False:
    maxEpochs = 5
    patience = 1

Create a Function, that given a PyTorch dataload with a set of data to train in, train a CNN model

In [13]:
def train_celltype_model(iteration, celltype_train_dataloader):
    # Define CNN with the loss and the optimizer
    net = PT_CNN_CellType()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)

    bestErrorDiff = 99999
    concurrentNonImproves = 0
    currentEpoch = 0

    bestValAcc = -1
    lstEpochs = []
    lstTrainAccs = []
    lstValAccs = []
    for epoch in range(maxEpochs):
        print("Starting Epoch " + str(epoch) + "...")
        currentEpoch = epoch

        # Set the Neural Network into training mode
        net.train()

        # Train through this epoch
        for i, data in enumerate(celltype_train_dataloader, 0):
            # Get the inputs
            inputs, labels = data

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Perform Forward and Backward propagation then optimize the weights
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        # Set the Neural Network into evaluation (test) mode, so we can evaluate both training and validation error
        net.eval()        
        trainingAccuracy, y_train_celltype, y_train_pred_celltype, y_train_pred_celltype_scores = predictCellTypeOnDataSetAccuracy(
            net, "Training", celltype_train_dataloader, True, False)
        validationAccuracy, y_val_celltype, y_val_pred_celltype, y_val_pred_celltype_scores  = predictCellTypeOnDataSetAccuracy(
            net, "Validation", celltype_val_dataloader, True, False)

        errorDiff = trainingAccuracy - validationAccuracy
        print("- Accuracy Difference: " + str(errorDiff))

        lstEpochs.append(epoch)
        lstTrainAccs.append(trainingAccuracy)
        lstValAccs.append(validationAccuracy)

        if epoch > 0 and (validationAccuracy - bestValAcc > 0.01):        
            # There is at least percentage point improvement in the validation F1, count this as a 
            # good iteration, regardless of the error difference
            print("- IsGoodStep")
            concurrentNonImproves = 0
            if errorDiff > 0 and errorDiff < bestErrorDiff:  
                bestErrorDiff = errorDiff        
        elif errorDiff < bestErrorDiff:        
            # This epoch is an improvement on the last, so we will continue. the concurrent non improve counts reset for the patience
            print("- IsBetter: " + str(errorDiff) + " : " + str(bestErrorDiff))        
            concurrentNonImproves = 0
            if errorDiff > 0:
                bestErrorDiff = errorDiff
        else:
            # This epoch has the same or worse performance than the last. Check if we have reached the patience, if so, then stop early
            print("- IsNotBetter: " + str(errorDiff) + " : " + str(bestErrorDiff))        
            concurrentNonImproves += 1
            if disableEarlyStopping == False:
                if concurrentNonImproves >= patience:
                    print("Early Stopping occurred at Epoch " + str(epoch))
                    break

        # update the val F1 score from the previous epoch if it's the best
        if validationAccuracy > bestValAcc:
            bestValAcc = validationAccuracy

    # Create a dataframe that can be used to plot the Training/Validation Loss plot, if we want to use it later
    dfLoss = pd.DataFrame({ 'epoch': lstEpochs, 'train': lstTrainAccs, 'validation': lstValAccs })

    # Once this is done, return the new CNN model and the loss dataframe
    return net, dfLoss


Now, load the initial training data into the celltype_training_data then configure it as a PyTorch Dataloader. This will be for our baseline model

Also, initialize a dataloader for the extra data

In [18]:
celltype_training_data = None

# Create copies of the Train and Extra images dataset, these will be continually updated through SS
dfImagesTrainSS = dfImagesTrain
dfExtraSS = dfImagesExtra

# Create a an initial dataloader for both train and extra images
if useFullData:
    celltype_training_data = CancerCellTypeDataset(isGoogleColab, dfImagesTrainSS, baseDirectory, transform=transform_normalize)
    celltype_extra_data = CancerCellTypeDataset(isGoogleColab, dfExtraSS, baseDirectory, transform=transform_normalize)
else:
    # For testing in a small dataset
    dfImagesTrainSS = dfImagesTrainSS.iloc[range(500), :].reset_index()
    dfExtraSS = dfExtraSS.iloc[range(1000), :].reset_index()
    celltype_training_data = CancerCellTypeDataset(isGoogleColab, dfImagesTrainSS, baseDirectory, transform=transform_normalize, target_transform=None)
    celltype_extra_data = CancerCellTypeDataset(isGoogleColab, dfExtraSS, baseDirectory, transform=transform_normalize, target_transform=None)

    
celltype_train_dataloader = DataLoader(celltype_training_data, batch_size=32, shuffle=True, num_workers=2)
celltype_extra_dataloader = DataLoader(celltype_extra_data, batch_size=32, shuffle=True, num_workers=2)

### Semi-supervised Loop

Here we will implement the Semi-supervised Learning Loop. This is the high level process:

<ol>
<li>Train a baseline model using the labelled Main Data</li>
<li>Predict using this model on the unlabelled Extra Data, taking care to also retain the Softmax score for each value, which is a probability of being that label</li>
<li>Review these predictions, find any that have a high confidence (generally 80% probability is used)</li>
<li>Take this data, and apply that predicted label, called a "pseudo-label". Incorporate this pseudo-labeled data into the training data (also remove them from the extra data)</li>
<li>Train an updated model using the updated training data</li>
<li>Iterate the process from Step 2, until no high confidence data is generated, or capped at a number of rounds</li>
</ol>

In [15]:
maxIterations = 5
currentIteration = 0
highConfThreshold = 0.8
noMorePseudos = False


In [17]:
print("Test")

# Using the current training dataloader, train a model
net, dfLoss = train_celltype_model(0, celltype_train_dataloader)

# Predict using the model on all the extra data. Ensure to get the predicted labels and the softmax probability score
extraAcc, y_extra_celltype, y_extra_pred_celltype, y_extra_pred_celltype_scores = predictCellTypeOnDataSetAccuracy(
    net, "Extra", celltype_extra_dataloader, True, True)



Test
Starting Epoch 0...
- Training F1: 0.574
- Validation F1: 0.6042677012609118
- Accuracy Difference: -0.030267701260911828
- IsBetter: -0.030267701260911828 : 99999
Starting Epoch 1...
- Training F1: 0.574
- Validation F1: 0.6042677012609118
- Accuracy Difference: -0.030267701260911828
- IsBetter: -0.030267701260911828 : 99999
Starting Epoch 2...
- Training F1: 0.574
- Validation F1: 0.6042677012609118
- Accuracy Difference: -0.030267701260911828
- IsBetter: -0.030267701260911828 : 99999
Starting Epoch 3...
- Training F1: 0.574
- Validation F1: 0.6042677012609118
- Accuracy Difference: -0.030267701260911828
- IsBetter: -0.030267701260911828 : 99999
Starting Epoch 4...
- Training F1: 0.574
- Validation F1: 0.6042677012609118
- Accuracy Difference: -0.030267701260911828
- IsBetter: -0.030267701260911828 : 99999
Starting Epoch 5...
- Training F1: 0.574
- Validation F1: 0.6042677012609118
- Accuracy Difference: -0.030267701260911828
- IsBetter: -0.030267701260911828 : 99999
Starting Ep

ValueError: Length of values (1000) does not match length of index (10384)

In [24]:

print(dfImagesTrainSS.shape)
print(dfExtraSS.shape)

(1000, 6)
(1000, 8)


In [23]:
for i in range(20):
    print(y_extra_pred_celltype_scores[i])

[1.1491333465297668e-14, 3.133239585210827e-14, 1.0, 1.0236431114644257e-13]
[3.2361017077847665e-12, 7.531211765332557e-12, 1.0, 2.0281824830714612e-11]
[9.122078634994646e-14, 2.2601531919890644e-13, 1.0, 6.840182717113286e-13]
[4.943386845701614e-12, 1.1122811872932292e-11, 1.0, 2.92586857519872e-11]
[4.363796218432724e-16, 1.35856228509004e-15, 1.0, 4.941116722864073e-15]
[2.143725362426397e-14, 5.879509796776275e-14, 1.0, 1.8644816051315016e-13]
[1.8943130729931908e-20, 7.505082722232072e-20, 1.0, 3.9656894552291563e-19]
[1.152315268147775e-15, 3.390747003241849e-15, 1.0, 1.1747698998212496e-14]
[5.873692070092067e-18, 2.045948030780291e-17, 1.0, 8.521272548619939e-17]
[5.324228877874246e-15, 1.494396511918239e-14, 1.0, 4.947985313033358e-14]
[1.703018079849013e-15, 4.8128297713541466e-15, 1.0, 1.6609116697003518e-14]
[4.279453640628816e-18, 1.5070344023099507e-17, 1.0, 6.507964568502623e-17]
[7.109092932384914e-18, 2.4237821315902203e-17, 1.0, 1.0186729905335026e-16]
[2.152701624

In [22]:
dfExtraPred = dfExtraSS
dfExtraPred["PredCellType"] = y_extra_pred_celltype
dfExtraPred["PredScoreTensor"] = y_extra_pred_celltype_scores

pred_scores = []
for i in range(len(y_extra_pred_celltype_scores)):
    # The predicted label will give the index of the related score
    ind = y_extra_pred_celltype[i]
    pred_scores.append(y_extra_pred_celltype_scores[i][ind])

    
dfExtraPred["PredScore"] = pred_scores

dfExtraPred.head(20)

,index,ImageName,isCancerous,cellType,trainValTest,PredCellType,PredScore,PredScoreTensor
0,0,./Image_classification_data/patch_images\10246...,0,-1,1,2,1.0,"[1.1491333465297668e-14, 3.133239585210827e-14..."
1,1,./Image_classification_data/patch_images\10247...,0,-1,0,2,1.0,"[3.2361017077847665e-12, 7.531211765332557e-12..."
2,2,./Image_classification_data/patch_images\10248...,0,-1,2,2,1.0,"[9.122078634994646e-14, 2.2601531919890644e-13..."
3,3,./Image_classification_data/patch_images\10249...,0,-1,0,2,1.0,"[4.943386845701614e-12, 1.1122811872932292e-11..."
4,4,./Image_classification_data/patch_images\10250...,0,-1,0,2,1.0,"[4.363796218432724e-16, 1.35856228509004e-15, ..."
5,5,./Image_classification_data/patch_images\10251...,0,-1,0,2,1.0,"[2.143725362426397e-14, 5.879509796776275e-14,..."
6,6,./Image_classification_data/patch_images\10252...,0,-1,1,2,1.0,"[1.8943130729931908e-20, 7.505082722232072e-20..."
7,7,./Image_classification_data/patch_images\10253...,0,-1,2,2,1.0,"[1.152315268147775e-15, 3.390747003241849e-15,..."
8,8,./Image_classification_data/patch_images\10254...,0,-1,2,2,1.0,"[5.873692070092067e-18, 2.045948030780291e-17,..."
9,9,./Image_classification_data/patch_images\10255...,0,-1,1,2,1.0,"[5.324228877874246e-15, 1.494396511918239e-14,..."


In [26]:
# Get the high Confidence predicted records and add them to the training data
dfHighConf = dfExtraPred[dfExtraPred["PredScore"] >= highConfThreshold]
dfHighConf = dfHighConf.drop(["PredCellType", "PredScore"], axis=1)

dfImagesTrainSS = dfImagesTrainSS.append(dfHighConf).reset_index(drop=True)

# Filter out the high conf records from the extra dataset
dfExtraSS = dfExtraPred[dfExtraPred["PredScore"] < highConfThreshold].reset_index()
dfExtraSS = dfExtraSS.drop(["PredCellType", "PredScore"], axis=1)

# Recreate the data loaders
celltype_train_dataloader = DataLoader(celltype_training_data, batch_size=32, shuffle=True, num_workers=2)
celltype_extra_dataloader = DataLoader(celltype_extra_data, batch_size=32, shuffle=True, num_workers=2)

print(dfImagesTrainSS.shape)
print(dfExtraSS.shape)

(2000, 7)
(0, 7)


In [ ]:

while (currentIteration < maxIterations) and noMorePseudos == False:
    print("Semi-Supervised Iteration " + str(currentIteration))
    
    # Using the current training dataloader, train a model
    print("Semi-Supervised Iteration " + str(currentIteration))
    net, dfLoss = train_celltype_model(currentIteration, celltype_train_dataloader)

    # Predict using the model on all the extra data. Ensure to get the predicted labels and the softmax probability score
    extraAcc, y_extra_celltype, y_extra_pred_celltype, y_extra_pred_celltype_scores = predictCellTypeOnDataSetAccuracy(
        net, "Extra", celltype_extra_dataloader, True, True)

    dfExtraPred = dfExtraSS
    dfExtraPred["PredCellType"] = y_extra_pred_celltype
    dfExtraPred["PredScore"] = y_extra_pred_celltype_scores


    # Get all the predictions that exceed the high confidence threshold
    # If there are none, set noMorePseudos = True then break the loop

    # Add the high conf records to the training data then update the dataloader

    # Filter out the high conf records from the extra dataset

    currentIteration += 1

In [ ]:
# dfLoss = pd.DataFrame({ 'epoch': lstEpochs, 'train': lstTrainAccs, 'validation': lstValAccs })
# graphutil.graphBasicTwoSeries(dfLoss, "epoch", "train", "validation", "CellType Training and Validation Accuracy", 
#         "Epoch", "Training Accuracy", "Validation Accuracy")

Predict on the Training Set to get the Training Accuracy and Error

In [ ]:
trainingAcc, y_train_celltype, y_train_pred_celltype, y_train_pred_celltype_scores = predictCellTypeOnDataSetAccuracy(
    net, "Training", celltype_train_dataloader, True, True)

In [ ]:
for i in range(5):
    print(y_train_pred_celltype_scores[i])

In [ ]:
a2util.getClassificationROC("CellType", "Training", y_train_celltype, y_train_pred_celltype, 4, y_train_pred_celltype_scores)

Predict on the Validation data and evaluate the results

In [ ]:
testAccuracy, y_test_celltype, y_test_pred_celltype, y_test_pred_celltype_scores  = predictCellTypeOnDataSetAccuracy(
        net, "Test", celltype_test_dataloader, True, True)


In [ ]:
a2util.getClassificationROC("CellType", "Test", y_test_celltype, y_test_pred_celltype, 4, y_test_pred_celltype_scores)

# Results

Append Results for both Binary IsCancerous and Cell Type here

### IsCancerous Results

Basic 01, 3 Layer NN with Dropout - On Colab, Full data - Previous best performing
- Training Accuracy: 0.942
- Training F1: 0.923
- Validation Accuracy: 0.869
- Validation F1: 0.897

CNN01, 3 Layer Classifier Binary with Sigmoid as Final Activation Function
- **Training**
- Accuracy Score: 0.9518948577261708
- Precision Score: 0.9426820475847152
- Recall Score: 0.9230497705612425
- F1 Score: 0.9327626181558766
- **Validation**
- Accuracy Score: 0.8894277400581959
- Precision Score: 0.9178981937602627
- Recall Score: 0.8972712680577849
- F1 Score: 0.9074675324675324

CNN02, 3 Layer Classifier with Early Stopping
- **Training**
- Accuracy Score: 0.9475564629322445
- Precision Score: 0.9328091493924232
- Recall Score: 0.9212848570420049
- F1 Score: 0.927011188066063
- **Test**
- Accuracy Score: 0.8764591439688716
- Precision Score: 0.8936507936507937
- Recall Score: 0.9036918138041734
- F1 Score: 0.8986432561851556


CNN02, 2 Layer Classifier with Early Stopping
- **Training**
- Accuracy Score: 0.9281612862064565
- Precision Score: 0.9235074626865671
- Recall Score: 0.8736321920225909
- F1 Score: 0.8978777435153275
- **Test**
- Accuracy Score: 0.8764591439688716
- Precision Score: 0.9039087947882736
- Recall Score: 0.8908507223113965
- F1 Score: 0.8973322554567501


### Cell type Results

Basic 01, 3 Layer NN with Dropout - On Colab, Full data - Previous best performing
- Training Accuracy: 0.844
- Training F1: 0.844
- Validation Accuracy: 0.778
- Validation F1: 0.778

CNN01, 3 Layer Classifier with Softmax as Final Activation Function, no Early Stopping
- **Training**
- Accuracy Score: 0.842414189102973
- Precision Score: 0.842414189102973
- Recall Score: 0.842414189102973
- F1 Score: 0.842414189102973
- **Validation**
- Accuracy Score: 0.7827352085354026
- Precision Score: 0.7827352085354026
- Recall Score: 0.7827352085354026
- F1 Score: 0.7827352085354026

CNN02, 3 Layer Classifier with Early Stopping
- **Training**
- Accuracy Score: 0.8100038279954064
- Precision Score: 0.8100038279954064
- Recall Score: 0.8100038279954064
- F1 Score: 0.8100038279954064
- **Validation**
- Accuracy Score: 0.7957198443579766
- Precision Score: 0.7957198443579766
- Recall Score: 0.7957198443579766
- F1 Score: 0.7957198443579766

CNN02, 2 Layer Classifier with Early Stopping
- **Training**
- Accuracy Score: 0.7962230445323466
- Precision Score: 0.7962230445323466
- Recall Score: 0.7962230445323466
- F1 Score: 0.7962230445323466
- **Test**
- Accuracy Score: 0.7928015564202334
- Precision Score: 0.7928015564202334
- Recall Score: 0.7928015564202334
- F1 Score: 0.7928015564202334

<h1>Analysis of Performance and Accuracy</h1>

<h3>Is Cancerous</h3>
<p>
The best performing model after initial CNN experiments, according to F1 Score, is the CNN with 3 Layer Classifier, with no Early stopping. the F1 Score is <strong>0.907</strong>, which is only a 0.01 improvement on the Basic NN 2 Layer model.
</p>
<p>
There isn't much benefit in the Early Stopping process here, looking at the 3 layer experiment with and without early stopping
</p>
<p>
However, we can see that the difference between the Training F1 and the Test F1 in this CNN is a lot smaller, with the Training F1 being 0.93. This is a situation where we have higher bias but less variance, with higher bias compared to other experiments. Therefore, it may be possible to model with more accuracy to reduce the bias, however, we must be careful not to overfit.
</p>

<h3>Cell Type</h3>
<p>
The best performing model after initial CNN experiments, according to Accuracy, is the CNN with 3 Layer Classifier, with Early stopping. the F1 Score is <strong>0.796</strong>, which is a 0.018 improvement on the Basic NN 2 Layer model.
</p>
<p>
However, we can see that the difference between the Training Accuracy and the Test Accuracy in this CNN very small, with the Testing accuracy being <strong>0.81</strong>. This indicates that the model is generalizing very well. However, it appears that this model has a higher bias than other experiments.
</p>

<h3>Conclusion</h3>
<p>
Therefore, for both models, it is worth experimenting on how accuracy can be improved. There are 3 methods that initially should be tried
</p>
<ol>
<li>First, experiment with a more complex CNN model, with additional complexity in the Convolution and the Classifier parts</li>
<li>Addition of more data to train with, specifically the Extra data provided</li>
<li>Data Augmentation</li>
</ol>

